# Week2 7월 10일 화요일 

# RDBMS, Sqlite

In [45]:
import sqlite3

In [52]:
conn = sqlite3.connect(":memory:")

In [53]:
cur = conn.cursor()

In [54]:
cur.executescript(
'''
        CREATE TABLE Artist(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
        
        CREATE TABLE Genre(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
        
        CREATE TABLE Album(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            artist_id INTEGER
        );
        
        CREATE TABLE Track(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            length INTEGER,
            rating INTERGER,
            count INTERGER,
            album_id INTEGER,
            genre_id INTEGER
        );
        
        INSERT INTO Artist (name) VALUES('IU');
        INSERT INTO Artist (name) VALUES('AKMU');
        
        INSERT INTO Genre (name) VALUES('Duet');
        INSERT INTO Genre (name) VALUES('Solo');

''')

In [55]:
cur.execute("SELECT * FROM Artist;")

In [56]:
print(cur.fetchall())

[(1, 'IU'), (2, 'AKMU')]


In [58]:
try :
    cur.executescript('''
        INSERT INTO Album (title, artist_id) VALUES('Pallete', 1);
        INSERT INTO Album (title, artist_id) VALUES('FourSpring', 2);
        
        INSERT INTO Track (title, album_id, genre_id) VALUES('Pallete', 1, 2);
        INSERT INTO Track (title, album_id, genre_id) VALUES('Go back home', 2, 1);
    
    ''')
except Exception as e :
    if conn :
        conn.rollback()
        
        print("Error:", e)

In [60]:
cur.execute("SELECT * FROM Album;")
print(cur.fetchall())

[(1, 'Pallete', 1), (2, 'FourSpring', 2)]


In [61]:
cur.execute("SELECT * FROM Track;")
print(cur.fetchall())

[(1, 'Pallete', None, None, None, 1, 2), (2, 'Go back home', None, None, None, 2, 1)]


In [62]:
# cross join 1
cur.execute("SELECT * FROM Artist, Album;")
print(cur.fetchall())

[(1, 'IU', 1, 'Pallete', 1), (1, 'IU', 2, 'FourSpring', 2), (2, 'AKMU', 1, 'Pallete', 1), (2, 'AKMU', 2, 'FourSpring', 2)]


In [63]:
# cross join 2
cur.execute("SELECT * FROM Artist join Album;")
print(cur.fetchall())

[(1, 'IU', 1, 'Pallete', 1), (1, 'IU', 2, 'FourSpring', 2), (2, 'AKMU', 1, 'Pallete', 1), (2, 'AKMU', 2, 'FourSpring', 2)]


In [64]:
# cross join 2
cur.execute("SELECT * FROM Artist cross join Album;")
print(cur.fetchall())

[(1, 'IU', 1, 'Pallete', 1), (1, 'IU', 2, 'FourSpring', 2), (2, 'AKMU', 1, 'Pallete', 1), (2, 'AKMU', 2, 'FourSpring', 2)]


In [66]:
# inner join
cur.execute('''
            SELECT at.name, ab.title
            FROM Artist as at JOIN Album as ab ON at.id = ab.artist_id;
            ''')
print(cur.fetchall())

[('IU', 'Pallete'), ('AKMU', 'FourSpring')]


In [67]:
# inner join
cur.execute('''
            SELECT ab.title, tr.title
            FROM Track as tr JOIN Album as ab ON tr.album_id = ab.id;
            ''')
print(cur.fetchall())

[('Pallete', 'Pallete'), ('FourSpring', 'Go back home')]


In [69]:
# inner join 3중 join
cur.execute('''
    select Artist.name, Album.title, Track.title, Genre.name
    from Track
    join Artist join Album join Genre on
    Track.album_id = Album.id
    and
    Track.genre_id = Genre.id
    and
    Album.artist_id = Artist.id;
''')
print(cur.fetchall())

[('IU', 'Pallete', 'Pallete', 'Solo'), ('AKMU', 'FourSpring', 'Go back home', 'Duet')]


In [70]:
# dump 쓰기
with open('dump.sql', 'w') as f :
    for line in conn.iterdump() :
        f.write('%s\n' % line) # 변수 출력하기 : %s 자리에 변수 들어감 

In [71]:
# dump 읽기
with open('dump.sql', 'r') as f :
    data = f.read()

print(data)

BEGIN TRANSACTION;
CREATE TABLE Album(
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            artist_id INTEGER
        );
INSERT INTO "Album" VALUES(1,'Pallete',1);
INSERT INTO "Album" VALUES(2,'FourSpring',2);
CREATE TABLE Artist(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
INSERT INTO "Artist" VALUES(1,'IU');
INSERT INTO "Artist" VALUES(2,'AKMU');
CREATE TABLE Genre(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
INSERT INTO "Genre" VALUES(1,'Duet');
INSERT INTO "Genre" VALUES(2,'Solo');
CREATE TABLE Track(
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            length INTEGER,
            rating INTERGER,
            count INTERGER,
            album_id INTEGER,
            genre_id INTEGER
        );
INSERT INTO "Track" VALUES(1,'Pallete',NULL,NULL,NULL,1,2);
INSERT INTO "Track" VALUES(2,'Go back home',NULL,NULL,NULL,2,1);
DELETE FROM "sql

In [73]:
# cursor 에서 dump 사용하기
conn1 = sqlite3.connect('dump.db')
cur = conn1.cursor()
cur.executescript(data)
cur.execute("select * from Artist;")
print(cur.fetchall())

[(1, 'IU'), (2, 'AKMU')]


In [74]:
cur.execute("insert into track (title) values ('baby')")

In [76]:
# left join 파이썬은 left join 만 지원함 (full, right X)
cur.execute("select track.title, album.title from track left join album on track.album_id = album.id")
print(cur.fetchall())

[('Pallete', 'Pallete'), ('Go back home', 'FourSpring'), ('baby', None)]


In [80]:
type(conn.iterdump())

generator

In [81]:
print(conn.iterdump())

<generator object _iterdump at 0x000000ED12BFDFC0>


In [82]:
print('\n'.join(conn.iterdump()))

BEGIN TRANSACTION;
CREATE TABLE Album(
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            artist_id INTEGER
        );
INSERT INTO "Album" VALUES(1,'Pallete',1);
INSERT INTO "Album" VALUES(2,'FourSpring',2);
CREATE TABLE Artist(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
INSERT INTO "Artist" VALUES(1,'IU');
INSERT INTO "Artist" VALUES(2,'AKMU');
CREATE TABLE Genre(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
INSERT INTO "Genre" VALUES(1,'Duet');
INSERT INTO "Genre" VALUES(2,'Solo');
CREATE TABLE Track(
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            length INTEGER,
            rating INTERGER,
            count INTERGER,
            album_id INTEGER,
            genre_id INTEGER
        );
INSERT INTO "Track" VALUES(1,'Pallete',NULL,NULL,NULL,1,2);
INSERT INTO "Track" VALUES(2,'Go back home',NULL,NULL,NULL,2,1);
DELETE FROM "sql

In [83]:
for line in conn.iterdump() :
    print(line)

BEGIN TRANSACTION;
CREATE TABLE Album(
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            artist_id INTEGER
        );
INSERT INTO "Album" VALUES(1,'Pallete',1);
INSERT INTO "Album" VALUES(2,'FourSpring',2);
CREATE TABLE Artist(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
INSERT INTO "Artist" VALUES(1,'IU');
INSERT INTO "Artist" VALUES(2,'AKMU');
CREATE TABLE Genre(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        );
INSERT INTO "Genre" VALUES(1,'Duet');
INSERT INTO "Genre" VALUES(2,'Solo');
CREATE TABLE Track(
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            length INTEGER,
            rating INTERGER,
            count INTERGER,
            album_id INTEGER,
            genre_id INTEGER
        );
INSERT INTO "Track" VALUES(1,'Pallete',NULL,NULL,NULL,1,2);
INSERT INTO "Track" VALUES(2,'Go back home',NULL,NULL,NULL,2,1);
DELETE FROM "sql